In [1]:
import sys, os
import numpy as np
import glob
import pandas as pd
import math

from scipy import stats
import matplotlib.pyplot as plt
from astropy.io import fits

from bokeh.io import output_notebook, show, export_png,export_svgs, save
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column,gridplot
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap

from ics.cobraCharmer import pfi as pfiControl
from ics.cobraCharmer import pfiDesign
from astropy.table import Table



In [39]:
def plotJ1OntimeSpeed(GroupIdx, dataFrame, xrange, yrange):

    mapper = Category20[len(GroupIdx)]
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    p = figure( tools=TOOLS, x_range=xrange, y_range=yrange,plot_height=500, plot_width=1000)

    colorcode = 0
    for i in GroupIdx:
        legendname="Fiber "+str(i)

        p.line(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_fwd'][dataFrame['fiberNo'] == i],\
        color=mapper[colorcode],line_width=2,legend=legendname)
        p.circle(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_fwd'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)
        p.line(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_rev'][dataFrame['fiberNo'] == i],color=mapper[colorcode],line_width=2)
        p.circle(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_rev'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)

        colorcode = colorcode + 1

    return p




class ontimeModel():
    
    def getThetaFwdSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid].loc[dataframe['J1_fwd'] > 0.001]

        onTimeArray = ndf['J1onTimeFwd'].values
        angSpdArray = ndf['J1_fwd'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def getThetaRevSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid].loc[dataframe['J1_rev'] < -0.001]

        onTimeArray = ndf['J1onTimeRev'].values
        angSpdArray = ndf['J1_rev'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def buildThetaModelfromDataFrame(self, dataframe, visibleFibers=False):

        # Reading all model and build a model list

        j1fwd_slope = np.zeros(57)
        j1fwd_int = np.zeros(57)
        
        j1rev_slope = np.zeros(57)
        j1rev_int = np.zeros(57)


        for pid in visibleFibers:
            fw_s, fw_i=self.getThetaFwdSlope(pid,dataframe)
            rv_s, rv_i=self.getThetaRevSlope(pid,dataframe)
            
            j1fwd_slope[pid-1] = fw_s
            j1fwd_int[pid-1] = fw_i
            
            j1rev_slope[pid-1] = rv_s
            j1rev_int[pid-1] = rv_i
       
        self.j1fwd_slope = j1fwd_slope
        self.j1fwd_int   = j1fwd_int
        self.j1rev_slope = j1rev_slope
        self.j1rev_int   = j1rev_int




In [16]:
path='/data/chyan/20190801/theta-ontime/'

In [17]:
brokens = [1,55]
visibles= [e for e in range(1,58) if e not in brokens]
goodIdx = np.array(visibles) - 1
goodGroupIdx = {}
for group in range(6):
    goodGroupIdx[group] = goodIdx[goodIdx%6==group] + 1

In [20]:
dataarray=[]
pidarray=[]
otfarray=[]
otrarray=[]
j1fwarray=[]
j1rvarray=[]

imageSet = ['run1', 'run2', 'run3', 'run4']

for i in imageSet:
    fw_file=path+f'{i}'+'/thetaSpeedFW.npy'
    rv_file=path+f'{i}'+'/thetaSpeedRV.npy'
    J1_fwd=np.load(fw_file)*180.0/math.pi 
    J1_rev=-np.load(rv_file)*180.0/math.pi
    
    xml = glob.glob(path+f'{i}'+'/*.xml')[0]
    model = pfiDesign.PFIDesign(xml)
    
    J1onTimeFwd = np.zeros(len(J1_fwd[goodIdx]))+model.motorOntimeSlowFwd1[goodIdx]*1000
    J1onTimeRev = np.zeros(len(J1_rev[goodIdx]))+model.motorOntimeSlowRev1[goodIdx]*1000
    pid=np.array(visibles)
    pidarray.append(pid)
    otfarray.append(J1onTimeFwd)
    otrarray.append(J1onTimeRev)
    
    j1fwarray.append(J1_fwd[goodIdx])
    j1rvarray.append(J1_rev[goodIdx])
    
d={'fiberNo': np.array(pidarray).flatten(),
   'J1onTimeFwd':  np.array(otfarray).flatten(), 'J1onTimeRev':  np.array(otrarray).flatten(),
    'J1_fwd': np.array(j1fwarray).flatten(), 'J1_rev': np.array(j1rvarray).flatten()}
df = pd.DataFrame(d)


In [99]:
otm = ontimeModel()
otm.buildThetaModelfromDataFrame(df,visibleFibers=visibles)

fwd_target = np.zeros(len(otm.j1fwd_int))+0.05
rev_target = np.zeros(len(otm.j1fwd_int))-0.05
newOntimeSlowFwd1 = (fwd_target-otm.j1fwd_int)/otm.j1fwd_slope
newOntimeSlowRev1 = (rev_target-otm.j1rev_int)/otm.j1rev_slope

newOntimeFwd1 = ((3.0*fwd_target)-otm.j1fwd_int)/otm.j1fwd_slope
newOntimeRev1 = ((3.0*rev_target)-otm.j1rev_int)/otm.j1rev_slope


newOntimeSlowFwd1[23] = 75.0
newOntimeSlowRev1[23] = 75.0

newOntimeSlowFwd1[29] = 50.0
newOntimeSlowRev1[29] = 60.0

newOntimeSlowFwd1[50] = 40.0


newOntimeSlowRev1[30] = 80.0


ind = np.where(newOntimeFwd1 > 80)
newOntimeFwd1[ind] = 80

ind = np.where(newOntimeRev1 > 80)
newOntimeRev1[ind] = 80
newOntimeRev1[20] = 75



/home/pfs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/home/pfs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/pfs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/home/pfs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [100]:
newOntimeSlowFwd1,newOntimeSlowRev1

(array([        inf, 30.95142327, 53.70196036, 34.47226362, 36.97742552,
        41.42799494, 91.3004803 , 54.47323957, 28.80237967, 42.2825548 ,
        85.8334726 , 38.85586353, 28.60293759, 38.40367299, 29.70947103,
        58.96847136, 58.26638051, 45.93910339, 34.88141115, 36.39591384,
        40.1261927 , 45.98636166, 32.86172175, 75.        , 48.7287471 ,
        38.42183964, 45.7491904 , 43.70688513, 49.13917301, 50.        ,
        39.78573324, 38.9382533 , 46.34624838, 37.05130876, 46.83421272,
        40.88026378, 53.3984201 , 39.85837672, 38.95827597, 51.07084511,
        39.14672016, 48.27121218, 36.82633577, 38.40821305, 47.7661768 ,
        33.37807803, 55.54580047, 75.63596913, 37.19123989, 45.20175837,
        40.        , 37.92683375, 34.19936545, 43.40472424,         inf,
        43.87358634, 60.23096295]),
 array([       -inf, 33.27552747, 49.03583904, 34.91844546, 48.60904128,
        47.53309812, 36.25948578, 46.52095709, 42.48029984, 46.89288717,
        30.1462

In [101]:
newOntimeFwd1,newOntimeRev1

(array([  80.        ,   78.65931533,   80.        ,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          70.82192485,   80.        ,   80.        ,   80.        ,
          73.79576427,   80.        ,   75.33620274,   80.        ,
          80.        ,   80.        ,   65.5270781 ,   80.        ,
          80.        ,   80.        ,   78.61393975,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          80.        ,   80.        ,   76.02408017,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          80.        ,   80.        ,   80.        ,   80.        ,
          80.        ,   80.        , -114.36317404,   80.        ,
          80.        ,   77.41127961,   80.        ,   80.        ,
          80.        ]),
 array([       -inf, 80

In [91]:
df.loc[df['fiberNo'] == 50]

,fiberNo,J1onTimeFwd,J1onTimeRev,J1_fwd,J1_rev
48,50,19.793882,19.793882,0.013539,-0.005032
103,50,24.793882,24.793882,0.015429,-0.009991
158,50,34.793882,34.793882,0.038158,-0.021108
213,50,39.793882,39.793882,0.039892,-0.026299


In [63]:
((3.0*rev_target[20])-otm.j1rev_int[20])/otm.j1rev_slope[20]

-68.93453497708629

In [27]:
TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

thetarange = tarray

p1 = plotJ1OntimeSpeed(goodGroupIdx[0], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p2 = plotJ1OntimeSpeed(goodGroupIdx[1], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p3 = plotJ1OntimeSpeed(goodGroupIdx[2], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p4 = plotJ1OntimeSpeed(goodGroupIdx[3], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p5 = plotJ1OntimeSpeed(goodGroupIdx[4], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p6 = plotJ1OntimeSpeed(goodGroupIdx[5], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])

x = np.array([])
y1 = np.array([])
y2 = np.array([])
for tms in thetarange:
    #print(np.mean(df['J2_fwd'][df['onTime']==tms].values))
    x=np.append(x,tms)
    y1=np.append(y1,np.median(df['J1_fwd'][df['J1onTime']==tms].values))
    y2=np.append(y2,np.median(df['J1_rev'][df['J1onTime']==tms].values))

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y1[:8])
#fit=np.polyfit(x, y*100, 1)
#print(fit)

q = figure(tools=TOOLS, x_range=[thetarange[0]-10, thetarange[-1]+10], y_range=[-0.5,0.5],plot_height=500, plot_width=1000)
q.circle(x=df['J1onTime'], y=df['J1_fwd'],radius=0.3,\
        color='red',fill_color=None)
q.circle(x=x,y=y1, radius=0.5, color='blue')
legendtext=f'Y={slope:.8f}X{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='blue',line_width=3, legend=legendtext)

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y2[:8])
#fit=np.polyfit(x, y*100, 1)
q.circle(x=df['J1onTime'], y=df['J1_rev'],radius=0.3,\
        color='green',fill_color=None)
q.circle(x=x,y=y2, radius=0.5, color='#3B0F6F')
legendtext=f'Y={slope:.8f}X+{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='#3B0F6F',line_width=3, legend=legendtext)

output_file("theta.html")
save(column(p1,p2,p3,p4,p5,p6,q), filename="theta.html", \
    title='Theta On-time')



'/Users/chyan/Documents/workspace/ics_cobraCharmer/notebooks/theta.html'

In [102]:

initXML = '/data/chyan/20190731/phi50Step/science15_phi50step_20190731.xml'
newXML = '/home/pfs/mhs/devel/ics_cobraCharmer/xml/Science15_thetaopt_20190731.xml'

model = pfiDesign.PFIDesign(initXML)
thetaTable = 'thetaOntime.csv'

OntimeFwd1=model.motorOntimeFwd1
OntimeRev1=model.motorOntimeRev1

SlowOntimeFwd1=model.motorOntimeSlowFwd1
SlowOntimeRev1=model.motorOntimeSlowRev1

OntimeFwd1[goodIdx] = newOntimeFwd1[goodIdx]/1000.0
OntimeRev1[goodIdx] = newOntimeRev1[goodIdx]/1000.0

SlowOntimeFwd1[goodIdx] = newOntimeSlowFwd1[goodIdx]/1000.0
SlowOntimeRev1[goodIdx] = newOntimeSlowRev1[goodIdx]/1000.0

pid=range(len(OntimeFwd1))
j1fwd_int = np.zeros(len(OntimeFwd1))
j1fwd_int[goodIdx] = otm.j1fwd_int[goodIdx]

j1fwd_slope = np.zeros(len(OntimeFwd1))
j1fwd_slope[goodIdx] = otm.j1fwd_slope[goodIdx]


j1rev_int = np.zeros(len(OntimeRev1))
j1rev_int[goodIdx] = otm.j1rev_int[goodIdx]
j1rev_slope = np.zeros(len(OntimeRev1))
j1rev_slope[goodIdx] = otm.j1rev_slope[goodIdx]



t=Table([pid, model.motorOntimeFwd1, j1fwd_int, j1fwd_slope, SlowOntimeFwd1,
         model.motorOntimeRev1,j1rev_int, j1rev_slope, SlowOntimeRev1],
         names=('Fiber No','Ori Fwd OT', 'FWD int', 'FWD slope', 'New Fwd OT',
                'Ori Rev OT', 'REV int', 'REV slope', 'New Rev OT'),
         dtype=('i2','f4', 'f4', 'f4','f4', 'f4', 'f4', 'f4', 'f4'))
t.write(thetaTable,format='ascii.ecsv',overwrite=True, 
        formats={'Fiber No':'%i','Ori Fwd OT': '%10.5f', 'FWD int': '%10.5f', 'FWD slope': '%10.5f', 'New Fwd OT': '%10.5f',\
        'Ori Rev OT': '%10.5f', 'REV int': '%10.5f', 'REV slope': '%10.5f', 'New Rev OT': '%10.5f'})



model.updateOntimes(thtFwd=SlowOntimeFwd1, thtRev=SlowOntimeRev1, fast=False)
model.updateOntimes(thtFwd=OntimeFwd1, thtRev=OntimeRev1, fast=True)

model.createCalibrationFile(newXML)

In [103]:
def plotBestThetaOnTimeFwd(DataFrame, OntimeModel, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo+1} Theta Fwd"
    p = figure(tools=TOOLS, x_range=[0,60], y_range=[0,0.4],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo+1]

    newOntimeFwd1 = (0.05-otm.j1fwd_int[fiberNo])/otm.j1fwd_slope[fiberNo]

    #print(newOntimeFwd1)
    x = range(5,70)
    y_predicted = [otm.j1fwd_slope[fiberNo]*i + otm.j1fwd_int[fiberNo]  for i in x]
    p.circle(x=newOntimeFwd1,y=[0.05],color='red', fill_color=None, radius=1.0)
    #p.circle(x=calibModel.motorOntimeSlowFwd2[fiberNo-1]*1000.0,y=speed_fwd[fiberNo-1]*180/math.pi,
    #color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J1onTimeFwd'],y=dd['J1_fwd'])
    p.line(x,y_predicted,color='red')
    
    return p

def plotBestThetaOnTimeRev(DataFrame, OntimeModel, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo} Theta Rev"
    p = figure(tools=TOOLS, x_range=[0,60], y_range=[-0.4,0],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo+1]

    newOntimeRev1 = (-0.05-otm.j1rev_int[fiberNo])/otm.j1rev_slope[fiberNo]


    x = range(5,70)
    y_predicted = [otm.j1rev_slope[fiberNo]*i + otm.j1rev_int[fiberNo]  for i in x]
    p.circle(x=newOntimeRev1,y=[-0.05],color='red', fill_color=None, radius=1.0)
    #p.circle(x=calibModel.motorOntimeSlowRev1[fiberNo-1]*1000.0,y=-speed_rev[fiberNo-1]*180/math.pi,
    #         color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J1onTimeRev'],y=dd['J1_rev'])
    p.line(x,y_predicted,color='red')
    
    return p



In [104]:
parray=[]

for f in goodIdx:
    p=plotBestThetaOnTimeFwd(df, otm,  f)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)

In [105]:
parray=[]

for f in goodIdx:
    p=plotBestThetaOnTimeRev(df, otm,  f)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)

In [87]:
otm.j1fwd_int

array([ 0.        , -0.01487695, -0.02057012, -0.01998227, -0.03583776,
       -0.01260907,  0.01672921, -0.01506313, -0.0185452 , -0.0239072 ,
        0.02239952, -0.02119592, -0.01329088, -0.0306287 , -0.01511418,
       -0.0083495 , -0.03037039, -0.04540512, -0.06382167, -0.02164871,
       -0.04979106, -0.00770518, -0.02182542, -0.82621128, -0.00905617,
       -0.02570573, -0.01862755, -0.02036258, -0.02231838, -0.72554555,
       -0.01192721, -0.02031332, -0.01548242, -0.01342602, -0.11044682,
       -0.01541415, -0.00984512, -0.03862257, -0.01338015, -0.04171573,
       -0.01301406, -0.03282251, -0.0155133 , -0.0322297 , -0.04362162,
       -0.01364874, -0.02592205, -0.27716659, -0.02236212, -0.01819431,
        0.04390259, -0.01934128, -0.01309363, -0.07763634,  0.        ,
       -0.02311673, -0.0528725 ])